In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from concurrent.futures import ThreadPoolExecutor, as_completed

# Encontrar todas as marcas
url = 'https://www.drogariasminasmais.com.br/medicamentos'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

marcas = soup.find_all("label", class_="vtex-checkbox__label w-100 c-on-base pointer")
marcas_corr = []
for marca in marcas:
    marcas_corr.append(str(marca).split('>')[1].split('<')[0])
for i in range(len(marcas_corr)):
    marcas_corr[i] = marcas_corr[i].lower().replace(" ", '-')
marcas_corr = marcas_corr[19:]

print(marcas_corr)

['disfunção-erétil', 'hypera', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'cimed', 'germed-generico', 'libbs', 'apsen', 'biolab-rx', 'supera', 'medley-genericos', 'momenta', 'farmoquimica', 'eurofarma-generico', 'myralis', 'abbott-rx', 'teuto-generico', 'ache-otc', 'sanofi-rx', 'cristalia/latinofarma', 'prati-donaduzzi-generico', 'torrent', 'sanofi-otc', 'uniao-quimica-similar', 'uniao-quimica-genom', 'legrand-generico', 'biolab-generico', 'neo-quimica-generico', 'pharlab', 'neo-quimica-similar', 'geolab-generico', 'nova-quimica-generico', 'gsk-otc', 'marjan', 'zanphy', 'novartis-rx', 'gsk-rx', 'uniao-quimica-generico', 'procter--gambl', 'biolab-avert-otc', 'merck-rx', 'alcon', 'servier', 'biosintetica-generico', 'allergan', 'teuto-similar', 'ems-similar', 'globo-generico', 'bayer-rx', 'cellera', 'astrazeneca', 'sandoz-generico', 'herbarium', 'ranbaxy', 'divcom', 'airela', 'zodiac-adium', 'sankyo-pharma', 'natulab', 'bayer-otc', 'boehringer', 'takeda', 'sigma-pharma-

In [18]:
def extrair_valores_por_chave(string, chave, categorico):
    valores = []  # Lista para armazenar os valores encontrados
    
    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')
    
    if categorico:
        # Enquanto houver ocorrências da chave
        while indice_chave != -1:
            # Encontrar o índice do próximo caractere após a ocorrência da chave
            indice_inicio_valor = indice_chave + len(f'"{chave}":') + 1
            
            # Encontrar o índice do próximo caractere que indica o final do valor
            indice_fim_valor = string.find('"', indice_inicio_valor)
            
            # Pegar o valor entre os índices de início e fim
            valor = string[indice_inicio_valor:indice_fim_valor]
            valores.append(valor)
            
            # Encontrar a próxima ocorrência da chave após a posição atual
            indice_chave = string.find(f'"{chave}":', indice_fim_valor)
        
    
    else:
        while indice_chave != -1:
                # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
                indice_inicio_valor = indice_chave + len(f'"{chave}":')
                
                # Encontrar o índice do próximo caractere que indica o final do preço
                indice_fim_valor = indice_inicio_valor
                while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
                    indice_fim_valor += 1
                
                # Pegar o valor entre os índices de início e fim
                valor = string[indice_inicio_valor:indice_fim_valor]
                valores.append(float(valor))
                
                # Encontrar a próxima ocorrência de '"Price":' após a posição atual
                indice_chave = string.find(f'"{chave}":', indice_fim_valor)
            
    return valores

# Exemplo de uso
texto = '... "productName":"Produto A", "productName":"Produto B", ...'
chave = "productName"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['Produto A', 'Produto B']

texto = '... "productReference":"123", "productReference":"45689", ...'
chave = "productReference"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['123', '45689']

texto = '... "Price":10.5, "Price":20.75, ...'
print(extrair_valores_por_chave(texto,"Price", False))  # Saída: [10.5, 20.75]

texto = '... "ean":"1234567890123", "ean":"9876543210987", ...'
print(extrair_valores_por_chave(texto,"ean",True))  # Saída: ['1234567890123', '9876543210987']


['Produto A', 'Produto B']
['123', '45689']
[10.5, 20.75]
['1234567890123', '9876543210987']


In [19]:
def extrair_categoria_por_chave(string, chave):
    valores = []  # Lista para armazenar os valores encontrados
    
    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(chave)
    # print(indice_chave)
    
    # Enquanto houver ocorrências da chave
    while indice_chave != -1:
        # Encontrar o índice do próximo caractere após a ocorrência da chave
        indice_inicio_valor = indice_chave + len(chave) + 0
        
        # Encontrar o índice do próximo caractere que indica o final do valor
        indice_fim_valor = string.find('\\', indice_inicio_valor)
        
        # Pegar o valor entre os índices de início e fim
        # print(string[indice_inicio_valor-len(chave):indice_fim_valor])
        valor = string[indice_inicio_valor:indice_fim_valor]

        if len(valor) > 35:
            valor = 'Mais Medicamentos'
            
        valores.append(valor)
        
        # Encontrar a próxima ocorrência da chave após a posição atual
        indice_chave = string.find(chave, indice_fim_valor)
            
    return valores

texto = 'com 30 Cápsulas - Hypera\",\"productReference\":\"31270\",\"linkText\":\"coledue-10mg-com-30-capsulas---hypera\",\"brand\":\"HYPERA\",\"brandId\":5,\"link\":\"\\u002Fcoledue-10mg-com-30-capsulas---hypera\\u002Fp\",\"categories\":{\"type\":\"json\",\"json\":[\"\\u002FMedicamentos\\u002FColesterol e Triglicerídeos\\u002F\",\"\\u002FMedicamentos\\u002F\"]},\"categoryId\":\"12\",\"releaseDate\":\"1709906100000\",\"priceRange\":{\"type\":\"id\",\"generated\":true,\"id\":\"$Product:sp-21093.priceRange\",\"typename\":\"ProductPriceRange\"},\"specificationGroups\":[{\"type\":\"id\",\"generated\":true,\"id\":\"Product:sp-21093.specificationGroups.0\",\"typename\":\"SpecificationGroup\"},{\"type\":\"id\",\"generated\":true,\"id\":\"Product:sp-21093.specificationGroups.1\",\"typename\":\"SpecificationGroup\"}],\"skuSpecifications\":[],\"productClusters'
print(extrair_categoria_por_chave(texto,'\"categories\":{\"type\":\"json\",\"json\":[\"\\u002FMedicamentos\\u002F'))  # Saída: ['1234567890123', '9876543210987']

['Colesterol e Triglicerídeos']


In [20]:



def extrair_quantidade(string):
    chave  = "recordsFiltered"

    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')

    # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
    indice_inicio_valor = indice_chave + len(f'"{chave}":')
    
    # Encontrar o índice do próximo caractere que indica o final do preço
    indice_fim_valor = indice_inicio_valor
    while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
        indice_fim_valor += 1
    
    # Pegar o valor entre os índices de início e fim
    quantidade = int(string[indice_inicio_valor:indice_fim_valor])
                    
    return quantidade


texto = '... "recordsFiltered":366, "ean":"9876543210987", ...'
print(extrair_quantidade(texto))  # Saída: ['1234567890123', '9876543210987']

366


In [21]:
colunas = ['Nome', 'Preco', 'Preco Sem Desconto', 'EAN', 'SKU', 'Marca','Categoria']

# Criar DataFrame vazio com as colunas especificadas
df = pd.DataFrame(columns=colunas)

# Mostrar DataFrame vazio
print(df)

Empty DataFrame
Columns: [Nome, Preco, Preco Sem Desconto, EAN, SKU, Marca, Categoria]
Index: []


In [22]:
# Função para processar uma página
def processar_pagina(marca, pagina):
    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=' + str(pagina)

    requisicao = requests.get(url)
    ean_lista = extrair_valores_por_chave(requisicao.text, "ean", True)
    sku_lista = extrair_valores_por_chave(requisicao.text, "productReference", True)
    nome_lista = extrair_valores_por_chave(requisicao.text, "productName", True)
    preco_lista = extrair_valores_por_chave(requisicao.text, 'Price', False)
    precosemdesconto_lista = extrair_valores_por_chave(requisicao.text, 'ListPrice', False)
    categoria_lista = extrair_categoria_por_chave(requisicao.text, '\"categories\":{\"type\":\"json\",\"json\":[\"\\u002FMedicamentos\\u002F')
    return ean_lista, sku_lista, nome_lista, preco_lista, precosemdesconto_lista, categoria_lista

In [23]:
ean_lista = []
sku_lista = []
nome_lista = []
preco_lista = []
precosemdesconto_lista = []
marca_lista = []
categoria_lista = []

In [25]:
executor = ThreadPoolExecutor(max_workers=10)
future_to_data = {}

for marca in marcas_corr:
    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1'

    requisicao = requests.get(url)
    quantidade = extrair_quantidade(requisicao.text)
    marca_lista.extend([marca] * quantidade)

    for pagina in range(1, math.ceil(quantidade/15) + 1):
        future = executor.submit(processar_pagina, marca, pagina)
        future_to_data[future] = (marca, pagina)

# Processar os resultados conforme estiverem prontos
for future in as_completed(future_to_data):
    marca, pagina = future_to_data[future]
    result_data = future.result()
    ean_lista.extend(result_data[0])
    sku_lista.extend(result_data[1])
    nome_lista.extend(result_data[2])
    preco_lista.extend(result_data[3])
    precosemdesconto_lista.extend(result_data[4])
    categoria_lista.extend(result_data[5])

executor.shutdown()



In [27]:
del marca_lista[:370]

df['Nome'] = nome_lista
df['Preco'] = preco_lista
df['Preco Sem Desconto'] = precosemdesconto_lista
df['EAN'] = ean_lista
df['SKU'] = sku_lista
df['Marca'] = marca_lista
df['Categoria'] = categoria_lista
df['Desconto'] = round((df['Preco Sem Desconto'] - df['Preco'])/df['Preco Sem Desconto'],2)

df.head()

,Nome,Preco,Preco Sem Desconto,EAN,SKU,Marca,Categoria,Desconto
0,Dioless 150mcg + 20mcg com 21 Comprimidos - Su...,33.72,44.33,7899420506253,8811,hypera,Mais Medicamentos,0.24
1,Zetsim 10\u002F40mg com 30 Comprimidos - Supera,142.91,177.35,7897337716888,8755,hypera,Mais Medicamentos,0.19
2,Nimegon Met 50\u002F1000mg com 56 Comprimidos ...,269.37,316.91,7897572004481,8669,hypera,Mais Medicamentos,0.15
3,Zetsim 10\u002F20mg com 30 Comprimidos - Supera,87.18,108.00,7897337716871,8647,hypera,Mais Medicamentos,0.19
4,"Minergi 0,375mg com 30 Comprimidos - Supera",72.36,91.23,7899420504723,8580,hypera,Mais Medicamentos,0.21


In [ ]:
df.to_excel('Drogaria_Minas_Mais_Excel.xlsx')